In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
connection = sqlite3.connect('shiptivity.db')

In [ ]:
feature_change_timestamp = 1527897600

query_before = f"""
SELECT
    DATE(login_timestamp, 'unixepoch') AS date,
    COUNT(DISTINCT user_id) AS daily_user_count
FROM login_history
WHERE login_timestamp <= 1527897600 -- Unix timestamp for 2018-06-02
GROUP BY date
ORDER BY date;
"""
df_before = pd.read_sql_query(query_before, connection)

query_after = f"""
SELECT
    DATE(login_timestamp, 'unixepoch') AS date,
    COUNT(DISTINCT user_id) AS daily_user_count
FROM login_history
WHERE login_timestamp > 1527897600 -- Unix timestamp for 2018-06-02
GROUP BY date
ORDER BY date;
"""
df_after = pd.read_sql_query(query_after, connection)